# [Ateliers: Technologies de l'intelligence Artificielle](https://github.com/wikistat/AI-Frameworks)

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# Traitement Naturel du Langage (NLP) : Génération de Texte avec des Réseaux Récurrent. 

Au cours de ce calepin, nous allons voir comment générer des description de produits à l'aide de Réseaux Récurents et notamment grace aux structure LSTM (Long-Short Term Memory). 

L'intérêt de cette application est limité. Les descriptions de textes de ce document sont trop pauvres syntaxiquement pour pouvoir juger réellement de la qualité du texte généré. L'intérêt réel de ce calepin est de voir comment les données doivent être mis en forme pour être utilisé dans un réseau recurrent dans un but de génération de texte.

## Librairies

In [1]:
#Importation des librairies utilisées

import pandas as pd
import numpy as np
import pickle
import functools
from tqdm import tqdm

import keras.models as km
import keras.layers as kl

Using TensorFlow backend.


## Téléchargement des données

In [2]:
DATA_DIR = ""
X = np.load(DATA_DIR+"data/description_coque.npy")
X.shape

(1761637,)

## Mise en forme  des données

### Création de la liste des caractères

In [3]:
chars = list(functools.reduce(lambda x,y : x.union(y), [set(x) for x in X], set()))
print(chars)

['J', 'u', '5', 'F', 'o', '2', '(', 'D', 'y', '…', 'i', 'h', 'R', ' ', '"', 'E', '!', 'ç', 'X', 'â', 'B', 'e', '.', 'z', '4', 'w', 'f', 'è', 'U', 'a', 'g', '6', '\xa0', 'M', 'O', '3', 'A', '&', 'à', ',', "'", 'l', 'm', '/', 'V', 'k', 'n', 'b', 'Y', '9', 'C', '*', 'q', 'ô', '1', 't', '%', '?', '8', 'j', 'S', '–', 'c', 'K', 'N', '-', 'v', 's', 'x', 'P', 'Q', 'p', 'é', 'W', 'L', 'T', ')', '+', 'd', 'Z', 'r', 'H', 'G', 'ê', 'I', '7', ':', '0']


In [4]:
chars.extend(["start","end"])

### Création des dictionnaires

In [5]:
int_to_char = {i:c for i,c in enumerate(chars)}
char_to_int = {c:i for i,c in int_to_char.items()}
I_START = char_to_int["start"]
I_END = char_to_int["end"]

### Paramètre du modèle 

In [6]:
SIZE_VOCAB = len(chars)
LENGTH_SEQUENCE = len(X[0])+1
N_X = 100000
print("Number of sentences : %d, Vocab size : %d, Sequence Length : %d" %(N_X, SIZE_VOCAB, LENGTH_SEQUENCE))

Number of sentences : 100000, Vocab size : 90, Sequence Length : 198


### Encodage des Descriptions


#### Définition des fonctions

In [7]:
def encode_input_output_sequence(x, length_sequence, size_vocab, char_to_int_dic, i_start, i_end):
    n = x.shape[0]
    x_vec = np.zeros((n,length_sequence, size_vocab))
    y_vec = np.zeros((n,length_sequence, size_vocab))
    x_vec[:,0,i_start] = 1
    y_vec[:,-1,i_end] = 1
    for ix,x in tqdm(enumerate(x)):
        for ic,c in enumerate(x):
            c_int = char_to_int_dic[c]
            x_vec[ix,ic+1,c_int]=1
    y_vec[:,:-1,:] = x_vec[:,1:,:] 
    return x_vec, y_vec


def decode_sequence(x, int_to_char_dic):
    seq = []
    for i in np.where(x)[1]:
        seq.append(int_to_char_dic[i])
    return "".join(seq)


#### Quelques exemples

In [9]:
x_vec, y_vec = encode_input_output_sequence(X[:N_X], LENGTH_SEQUENCE, SIZE_VOCAB, char_to_int,I_START,I_END)

100000it [00:18, 5392.72it/s]


In [10]:
print(decode_sequence(x_vec[8], int_to_char))
print(decode_sequence(y_vec[8], int_to_char))


startPour samsung galaxy s2 - i9100 : coque decor drapeau us vintage - Cette coque fantaisie protège et habille votre SAMSUNG Galaxy S2 - i9100. Parfaitement adaptée, elle permet l… Voir la présentation
Pour samsung galaxy s2 - i9100 : coque decor drapeau us vintage - Cette coque fantaisie protège et habille votre SAMSUNG Galaxy S2 - i9100. Parfaitement adaptée, elle permet l… Voir la présentationend


**Exercice** Retrouvez la phrase originale à partir de la phrase encodé. Vérifiez que x et y sont bien les mêmes fonciton d'écalé d'un index

## Apprentissage

In [11]:
nb_hidden = 32
epochs = 20
batch_size=128

model = km.Sequential()
model.add(kl.LSTM(nb_hidden, input_shape=(None, SIZE_VOCAB), return_sequences=True))
model.add(kl.TimeDistributed(kl.Dense(SIZE_VOCAB)))
model.add(kl.Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 32)          15744     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 90)          2970      
_________________________________________________________________
activation_1 (Activation)    (None, None, 90)          0         
Total params: 18,714
Trainable params: 18,714
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.fit(x_vec, y_vec, epochs=epochs, batch_size=batch_size)

Epoch 1/20
 20992/100000 [=====>........................] - ETA: 15:40 - loss: 3.3456

KeyboardInterrupt: 